In [1]:
import scanpy as sc
import numpy as np
from pathlib import Path
from tqdm.auto import tqdm
import networkx as nx
import pickle
import pandas as pd
import newick

In [2]:
root = Path("/storage/groups/ml01/datasets/raw/2021-11-13_hu_heart_regeneration_marius.lange")
ls = root / "loom_files"
looms = !ls $ls
looms = list(looms)
!ls $root

loom_files  meta  README.txt  zips


In [3]:
meta = pd.read_csv(root / "meta" / "final_metadata_Tmacromerged_2.csv")
meta

,Cell,orig.ident,nCount_RNA,nFeature_RNA,percent.mito,time,morphine,AV,inhib,so.far.ident.subtypes,...,paper.ident,RNA_snn_res.2,RNA_snn_res.3,RNA_snn_res.3.5,work.ident2,work.ident3,final.zoom,lineage.ident,Cell_type,Cell_name
0,H5_AAACCTGAGACTACAA,H5,1112,337,17.535971,Ctrl,Ctrl,Wholeheart,NaN,Atrium,...,Cardiomyocytes,19,10,10,Cardiomyocytes,Cardiomyocytes A,Cardiomyocytes A,Cardiomyocytes A,Cardiomyocytes (Atrium),ndH5_AAACCTGAGACTACAA
1,H5_AAACGGGAGGCGCTCT,H5,834,366,6.714628,Ctrl,Ctrl,Wholeheart,NaN,Atrium,...,Cardiomyocytes,14,35,27,Cardiomyocytes,Cardiomyocytes A,Cardiomyocytes A,Cardiomyocytes A,Cardiomyocytes (Atrium),ndH5_AAACGGGAGGCGCTCT
2,H5_AAAGATGAGATCCCAT,H5,1604,510,1.807980,Ctrl,Ctrl,Wholeheart,NaN,Atrium,...,Cardiomyocytes,31,23,27,Cardiomyocytes,Cardiomyocytes A,Cardiomyocytes A,Cardiomyocytes A,Cardiomyocytes (Atrium),ndH5_AAAGATGAGATCCCAT
3,H5_AAAGATGCATCAGTCA,H5,3244,577,4.562269,Ctrl,Ctrl,Wholeheart,NaN,Atrium,...,Cardiomyocytes,14,23,27,Cardiomyocytes,Cardiomyocytes A,Cardiomyocytes A,Cardiomyocytes A,Cardiomyocytes (Atrium),ndH5_AAAGATGCATCAGTCA
4,H5_AAAGATGTCGATAGAA,H5,1079,426,7.321594,Ctrl,Ctrl,Wholeheart,NaN,Atrium,...,Cardiomyocytes,14,35,27,Cardiomyocytes,Cardiomyocytes A,Cardiomyocytes A,Cardiomyocytes A,Cardiomyocytes (Atrium),ndH5_AAAGATGTCGATAGAA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202070,Hr34_TAGATCGTCTTCGCTG,Hr34,13868,2726,2.531007,3dpi,Morph,Wholeheart,IWR1,NaN,...,Proliferating cells,46,73,82,Proliferating cells,Proliferating cells,Proliferating cells,Proliferating cells,Proliferating cells,ndHr34_TAGATCGTCTTCGCTG
202071,Hr35_CTACAGAAGTGAGGTC,Hr35,11595,2378,3.311772,3dpi,Morph,Wholeheart,IWR1,NaN,...,Proliferating cells,46,73,65,Immune Cells,Proliferating cells,Macrophage (Fibroblast duplex),Macrophage (Fibroblast duplex),Macrophages,ndHr35_CTACAGAAGTGAGGTC
202072,Hr35_GGCTTGGCACACCAGC,Hr35,30883,3903,2.700515,3dpi,Morph,Wholeheart,IWR1,NaN,...,Proliferating cells,46,79,1,Immune Cells,Smooth muscle cells,Macrophage (Fibroblast duplex),Macrophage (Fibroblast duplex),Macrophages,ndHr35_GGCTTGGCACACCAGC
202073,Hr35_GGTGTCGGTTATGGTC,Hr35,13746,2980,4.394006,3dpi,Morph,Wholeheart,IWR1,NaN,...,Proliferating cells,46,73,82,Proliferating cells,Proliferating cells,Proliferating cells,Proliferating cells,Proliferating cells,ndHr35_GGTGTCGGTTATGGTC


In [4]:
adatas = {}
for loom in tqdm(sorted(looms)):
    adata = sc.read_loom(ls / loom)
    # np.testing.assert_allclose(adata.X.data, adata.layers['matrix'].data)
    adata.var_names_make_unique()
    adatas[str(loom).split(".")[0]] = adata

  0%|          | 0/43 [00:00<?, ?it/s]

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Vari

In [5]:
for k, adata in tqdm(adatas.items()):
    # adata = adata.copy()
    lib = k.split("_")[0]
    tmp_meta = meta[meta['orig.ident'] == lib]
    if not len(tmp_meta):
        bc = adata.obs_names.str.split(":").str[1].str.rstrip("x")
        adata.obs_names = pd.Series([lib] * len(bc)).str.cat(bc, sep="_")
        adata.obs['orig.ident'] = lib
        print(k, lib)
        continue
        
    sanity_adata = adata.obs_names.str.split(":").str[0].str.split("_").str[0]
    sanity_meta = tmp_meta['Cell'].str.split("_").str[0]
    np.testing.assert_array_equal(sanity_adata, [lib] * adata.n_obs)
    np.testing.assert_array_equal(sanity_meta, [lib] * len(sanity_meta))
    
    bc = adata.obs_names.str.split(":").str[1].str.rstrip("x")
    adata.obs_names = pd.Series([lib] * len(bc)).str.cat(bc, sep="_")
    tmp_obs = pd.merge(adata.obs, tmp_meta, left_index=True, right_on='Cell', how='left')
    np.testing.assert_array_equal(adata.obs_names, tmp_obs['Cell'])
    adata.obs = tmp_obs.set_index("Cell")
    cn = adata.obs['Cell_name'].str[2:]
    mask = ~(cn.isnull())
    np.testing.assert_array_equal(adata.obs_names[mask], cn[mask])
    del adata.obs['Cell_name']
    #assert len(tmp_meta), lib

  0%|          | 0/43 [00:00<?, ?it/s]

Hr16_v3Dr11 Hr16
Hr17_v3Dr11 Hr17
Hr18_v3Dr11 Hr18
Hr5_v3Dr11 Hr5


In [6]:
ads = [a for a in adatas.values()]
len(ads)

43

In [7]:
res = ads[0].concatenate(ads[1:])
res

AnnData object with n_obs × n_vars = 521948 × 31466
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mito', 'time', 'morphine', 'AV', 'inhib', 'so.far.ident.subtypes', 'so.far.ident', 'RNA_snn_res.4', 'seurat_clusters', 'work.ident', 'fibro.subtypes', 'is.inhib', 'split.inhib', 'zoom.ident', 'zoom.subtypes', 'plot.ident', 'CM.ident', 'paper.ident', 'RNA_snn_res.2', 'RNA_snn_res.3', 'RNA_snn_res.3.5', 'work.ident2', 'work.ident3', 'final.zoom', 'lineage.ident', 'Cell_type', 'batch'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    layers: 'matrix', 'ambiguous', 'spliced', 'unspliced'

In [8]:
res.obs['orig.ident'].astype("category")

H5_AACCGCGAGTCCCACG-0       H5
H5_AACCGCGAGACGCACA-0       H5
H5_AAACCTGAGCACGCCT-0       H5
H5_AAAGATGAGTAGCCGA-0       H5
H5_AAACCTGAGACTACAA-0       H5
                          ... 
Hr9_TTTGTCATCCTTGGTC-42    NaN
Hr9_TTTGGTTTCGTGGACC-42    NaN
Hr9_TTTGGTTAGGCCATAG-42    Hr9
Hr9_TTTGCGCTCGACAGCC-42    NaN
Hr9_TTTGGTTTCTGTACGA-42    NaN
Name: orig.ident, Length: 521948, dtype: category
Categories (43, object): ['H5', 'H6', 'H7', 'H8a', ..., 'Hr7a', 'Hr7v', 'Hr8', 'Hr9']

In [9]:
t = res.obs['orig.ident'].astype("category").fillna(pd.Series(res.obs_names.str.split("_").str[0], index=res.obs_names))
res.obs['orig.ident'] = t
res.obs['orig.ident']

H5_AACCGCGAGTCCCACG-0       H5
H5_AACCGCGAGACGCACA-0       H5
H5_AAACCTGAGCACGCCT-0       H5
H5_AAAGATGAGTAGCCGA-0       H5
H5_AAACCTGAGACTACAA-0       H5
                          ... 
Hr9_TTTGTCATCCTTGGTC-42    Hr9
Hr9_TTTGGTTTCGTGGACC-42    Hr9
Hr9_TTTGGTTAGGCCATAG-42    Hr9
Hr9_TTTGCGCTCGACAGCC-42    Hr9
Hr9_TTTGGTTTCTGTACGA-42    Hr9
Name: orig.ident, Length: 521948, dtype: category
Categories (43, object): ['H5', 'H6', 'H7', 'H8a', ..., 'Hr7a', 'Hr7v', 'Hr8', 'Hr9']

In [10]:
del res.obs['batch']
assert len(res.obs_names.str.split("-").str[0].unique()) == res.n_obs
res.obs_names = res.obs_names.str.split("-").str[0]
res.obs_names

Index(['H5_AACCGCGAGTCCCACG', 'H5_AACCGCGAGACGCACA', 'H5_AAACCTGAGCACGCCT',
       'H5_AAAGATGAGTAGCCGA', 'H5_AAACCTGAGACTACAA', 'H5_AAATGCCTCCGATATG',
       'H5_AACCGCGCAGACGCTC', 'H5_AAATGCCGTACCGCTG', 'H5_AAACGGGAGGGAAACA',
       'H5_AAATGCCGTACCGGCT',
       ...
       'Hr9_TTTGGTTTCTTTAGTC', 'Hr9_TTTGCGCGTGGTGTAG', 'Hr9_TTTGTCATCTCACATT',
       'Hr9_TTTGGTTAGTACGCCC', 'Hr9_TTTGTCACAAGCGATG', 'Hr9_TTTGTCATCCTTGGTC',
       'Hr9_TTTGGTTTCGTGGACC', 'Hr9_TTTGGTTAGGCCATAG', 'Hr9_TTTGCGCTCGACAGCC',
       'Hr9_TTTGGTTTCTGTACGA'],
      dtype='object', length=521948)

In [11]:
trees = pd.read_csv(root / "meta" / "All_sbcf_edgelists.csv")
trees

,from,to,Cell.type,heart
0,nd0,nd0_1,NaN,H5
1,nd0,nd0_2,NaN,H5
2,nd0,nd0_3,NaN,H5
3,nd0,ndH5_AAACGGGAGCCTTGAT,Endocardium (Ventricle),H5
4,nd0,ndH5_AAACGGGGTAGAGTGC,Smooth muscle cells,H5
...,...,...,...,...
51543,nd0_2_1_1_2,ndHr27_TCCCACAAGCGTGTCC,Endocardium (Ventricle),Hr27
51544,nd0_2_1_1_2,ndHr27_TGACGCGTCGCTCTCA,Endocardium (Ventricle),Hr27
51545,nd0_2_1_1_2,ndHr27_TGAGACTGTACCTAGT,Endocardium (Ventricle),Hr27
51546,nd0_2_1_1_2,ndHr27_TGCGATAGTTACGTAC,Valve fibroblasts,Hr27


In [12]:
trees['from'] = trees['from'].str.replace("^nd", "", regex=True)
trees['to'] = trees['to'].str.replace("^nd", "", regex=True)
trees['Cell_type'] = trees.pop('Cell.type').astype("category")
trees['heart'] = trees['heart'].astype("category")
trees

,from,to,heart,Cell_type
0,0,0_1,H5,NaN
1,0,0_2,H5,NaN
2,0,0_3,H5,NaN
3,0,H5_AAACGGGAGCCTTGAT,H5,Endocardium (Ventricle)
4,0,H5_AAACGGGGTAGAGTGC,H5,Smooth muscle cells
...,...,...,...,...
51543,0_2_1_1_2,Hr27_TCCCACAAGCGTGTCC,Hr27,Endocardium (Ventricle)
51544,0_2_1_1_2,Hr27_TGACGCGTCGCTCTCA,Hr27,Endocardium (Ventricle)
51545,0_2_1_1_2,Hr27_TGAGACTGTACCTAGT,Hr27,Endocardium (Ventricle)
51546,0_2_1_1_2,Hr27_TGCGATAGTTACGTAC,Hr27,Valve fibroblasts


In [29]:
def create_graph(g):
    G = nx.DiGraph()
    for _, row in g.iterrows():
        G.add_edge(row['from'], row['to'], cell_type=row['Cell_type'])
    return G

def tree_to_newick(g, root=None):
    if root is None:
        roots = list(filter(lambda p: p[1] == 0, g.in_degree()))
        assert 1 == len(roots)
        root = roots[0][0]
    subgs = []
    for child in g[root]:
        if len(g[child]) > 0:
            subgs.append(tree_to_newick(g, root=child))
        else:
            subgs.append(child)
    return "(" + ','.join(subgs) + ")"

In [14]:
graphs = trees.groupby("heart").apply(create_graph).to_dict()

In [15]:
for k in graphs:
    if k not in res.obs['orig.ident'].cat.categories:
        print(k)

H8
Hr2
Hr6
Hr7


In [16]:
for lib, g in graphs.items():
    cnt = 0
    for n in g.nodes:
        if f"{lib}_" in n:
            cnt += n in res.obs_names
    for (s, t) in g.edges:
        if f"{lib}_" in s and s in res.obs_names:
            ct = g.edges[s, t]['cell_type']
            assert res.obs.loc[s]['Cell_type'] == ct
        if f"{lib}_" in t and t in res.obs_names:
            ct = g.edges[s, t]['cell_type']
            assert res.obs.loc[t]['Cell_type'] == ct
            
    print(cnt, len(graphs[lib]), lib, lib in res.obs['orig.ident'].cat.categories)
    if not cnt:
        assert lib in ['H8', 'Hr2', 'Hr6', 'Hr7']

1036 1055 H5 True
1228 1237 H6 True
2194 2211 H7 True
0 4949 H8 False
3548 3563 Hr1 True
530 551 Hr10 True
1073 1080 Hr11 True
2389 2402 Hr12 True
801 809 Hr13 True
1014 1035 Hr14 True
643 651 Hr15 True
108 114 Hr19 True
0 9695 Hr2 False
249 259 Hr20 True
235 245 Hr21 True
645 659 Hr22 True
312 325 Hr23 True
1081 1095 Hr24 True
607 624 Hr25 True
4216 4232 Hr26 True
2930 2948 Hr27 True
2658 2681 Hr3 True
1993 2007 Hr4 True
0 3804 Hr6 False
0 3342 Hr7 False


In [30]:
res.uns['trees'] = {lib:tree_to_newick(g) + ";" for lib, g in graphs.items()}

In [ ]:
sc.write("adata.h5ad", res)